In [ ]:
import itertools

class FireEvent:
    generate_id = itertools.count(1, 1)

    def __init__(self, initial_size, initial_intensity):
        self.event_id = next(FireEvent.generate_id)
        self.size = initial_size  # Size of the fire in square hectares
        self.intensity = initial_intensity  # Intensity of the fire

    def scale_fire_size(self, wait_time):
        """
        Scale fire size based on wait time in the queue.
        Growth is proportional to the intensity, size, and waiting time.
        """
        if wait_time > 0:
            growth_rate = (self.intensity) * wait_time * (1 + (self.size / 200))
            self.size += growth_rate

    def get_service_time(self):
        #base time is 30 mins for every fire
        base_time = 1/48
        #size factor relates to size in some way
        size_factor = self.size / 100
        #intensity increases time to put out
        #intensity should be 0-1.0 quantity
        intensity_factor = self.intensity + 1
        if self.intensity >= 0.9:
          intensity_factor = 3


        if self.size > 0:
            service_time = (base_time + size_factor) * intensity_factor  # Adjust scaling factor if needed
            return service_time
        return 0

    def __str__(self):
        return f"FireEvent ID: {self.event_id}, Size: {self.size:.2f} hectares, Intensity: {self.intensity:.2f}"



In [ ]:
import numpy as np

def generate_fire_events(n,seed=10):
  """
  Generate all stats for fire simulation given size n
  """
  #set seed
  np.random.seed(seed)
  #reset itertools
  generate_id = itertools.count(1, 1)
  #not sure whether to do 1/lambda or lambda
  arr_lambda = 1/0.44
  arr_lambda = arr_lambda / 10

  #shaping parameters for pareto distribution
  a, m = 0.42, 2.10
  fire_sizes = (np.random.pareto(a=a, size=n) + 1) * m
  fire_sizes = np.minimum(fire_sizes, 250000)

  fire_intensities = [np.random.exponential(scale=1.0) for _ in range(n)]
  fire_intensities = [value / (value+1) for value in fire_intensities]

  #interarrival times - based on inverse lambda from poisson dist
  inter_arrival_time = [np.random.exponential(scale=arr_lambda) for _ in range(n)]

  fires = [FireEvent(fire_sizes[i],fire_intensities[i]) for i in range(n)]
  return fires, inter_arrival_time




In [ ]:
value = np.random.exponential(scale=1.0)
scaled_value = value / (value + 1)  # Map values to (0, 1)

print(scaled_value)

0.7003259776127412


In [ ]:
def generate_suppression_time(event):
  #base time is 6 hours
  base_time = 0.25
  size_factor = event.size/50
  intensity_factor = 1
  return (base_time+size_factor)*intensity_factor

In [ ]:
#testing generate_fire_events & service times
fires, inter_arrival_times = generate_fire_events(50)
for fire in fires:
  print(fire, "; ", fire.get_service_time())

for iar in inter_arrival_times:
  print(iar)

ev = FireEvent(20, 0.95)
print(ev, "; ", ev.get_service_time())

FireEvent ID: 1, Size: 70.45 hectares, Intensity: 0.33 ;  0.9665862195841356
FireEvent ID: 2, Size: 2.21 hectares, Intensity: 0.09 ;  0.04674247195950027
FireEvent ID: 3, Size: 22.94 hectares, Intensity: 0.63 ;  0.4084699193677682
FireEvent ID: 4, Size: 56.33 hectares, Intensity: 0.14 ;  0.666409976937141
FireEvent ID: 5, Size: 10.86 hectares, Intensity: 0.33 ;  0.17170299514913354
FireEvent ID: 6, Size: 3.85 hectares, Intensity: 0.74 ;  0.10341082857737657
FireEvent ID: 7, Size: 3.55 hectares, Intensity: 0.81 ;  0.10225317938521368
FireEvent ID: 8, Size: 63.12 hectares, Intensity: 0.38 ;  0.8989754237785276
FireEvent ID: 9, Size: 3.26 hectares, Intensity: 0.64 ;  0.08750170235072811
FireEvent ID: 10, Size: 2.62 hectares, Intensity: 0.22 ;  0.05756008280399051
FireEvent ID: 11, Size: 32.95 hectares, Intensity: 0.48 ;  0.5172709556480781
FireEvent ID: 12, Size: 3108.72 hectares, Intensity: 0.70 ;  52.87796493271182
FireEvent ID: 13, Size: 2.12 hectares, Intensity: 0.43 ;  0.060246803755

In [ ]:
# Queuing system with fire object. Currently foramtting is iffy due to values not being rounded
def sim_fires_fifo(size, fires,inter_arrival_time):
  # Calculate arrival time
  arrival_time = [0 for i in range(size)]

  # initial
  arrival_time[0] = 0

  for i in range(1,size):
    arrival_time[i] = inter_arrival_time[i]+arrival_time[i-1]


  Time_Service_Begin = [0 for i in range(size)]
  Time_Customer_Waiting_in_Queue = [0 for i in range(size)]
  Time_Service_Ends = [0 for i in range(size)]
  Time_Customer_Spend_in_System = [0 for i in range(size)]
  System_idle = [0 for i in range(size)]

  Time_Service_Begin[0] = arrival_time[0]
  Time_Service_Ends[0] = fires[0].get_service_time()
  Time_Customer_Spend_in_System[0] = fires[0].get_service_time()

  Time_Service_Begin[1] = fires[1].get_service_time() # This is how it was, seems wrong
  Time_Service_Ends[1] = fires[1].get_service_time()
  Time_Customer_Spend_in_System[1] = fires[1].get_service_time()


  Time_Service_Begin[2] = arrival_time[2]
  Time_Service_Ends[2] = fires[2].get_service_time()
  Time_Customer_Spend_in_System[2] = fires[2].get_service_time()


  for i in range(3,size):
    # Time Service Begin
    if (arrival_time[i] >= Time_Service_Ends[i-1] and arrival_time[i] >= Time_Service_Ends[i-2] and arrival_time[i] >= Time_Service_Ends[i-3]):
        Time_Service_Begin[i] = arrival_time[i]
    else:
        if (arrival_time[i] <= Time_Service_Ends[i-1] and arrival_time[i] <= Time_Service_Ends[i-2] and arrival_time[i] <= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = min(Time_Service_Ends[i-1], Time_Service_Ends[i-2], Time_Service_Ends[i-3])

        elif (arrival_time[i] <= Time_Service_Ends[i-1] and arrival_time[i] <= Time_Service_Ends[i-2] and arrival_time[i] >= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = min(Time_Service_Ends[i-1], Time_Service_Ends[i-2])

        elif (arrival_time[i] <= Time_Service_Ends[i-1] and arrival_time[i] >= Time_Service_Ends[i-2] and arrival_time[i] <= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = min(Time_Service_Ends[i-1], Time_Service_Ends[i-3])

        elif (arrival_time[i] >= Time_Service_Ends[i-1] and arrival_time[i] <= Time_Service_Ends[i-2] and arrival_time[i] <= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = min(Time_Service_Ends[i-2], Time_Service_Ends[i-3])

        elif (arrival_time[i] <= Time_Service_Ends[i-1] and arrival_time[i] >= Time_Service_Ends[i-2] and arrival_time[i] >= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = Time_Service_Ends[i-1]

        elif (arrival_time[i] >= Time_Service_Ends[i-1] and arrival_time[i] <= Time_Service_Ends[i-2] and arrival_time[i] >= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = Time_Service_Ends[i-2]

        elif (arrival_time[i] >= Time_Service_Ends[i-1] and arrival_time[i] >= Time_Service_Ends[i-2] and arrival_time[i] <= Time_Service_Ends[i-3]):
            Time_Service_Begin[i] = Time_Service_Ends[i-3]

    # Time customer waiting in queue
    Time_Customer_Waiting_in_Queue[i] = Time_Service_Begin[i]-arrival_time[i]

    if(Time_Customer_Waiting_in_Queue[i] > 0):
      fires[i].scale_fire_size(Time_Customer_Waiting_in_Queue[i])

    # Time service ends
    Time_Service_Ends[i] = Time_Service_Begin[i] + fires[i].get_service_time()

    # Time Customer Spend in the system
    Time_Customer_Spend_in_System[i] = Time_Service_Ends[i] - arrival_time[i]

    # Time when system remains idle
    if (arrival_time[i]>Time_Service_Ends[i-1]):
      System_idle[i] = arrival_time[i]-Time_Service_Ends[i-1]
    else:
      System_idle[i] = 0

  service_times = [fire.get_service_time() for fire in fires]

  from prettytable import PrettyTable

  x = PrettyTable()

  fire_ids = [fire.event_id for fire in fires]

  column_names = ['Fire','IAT','AT','ST','TSB','TCWQ','TSE','TCSS','System Idle']
  data = [fire_ids,inter_arrival_time,arrival_time, service_times, Time_Service_Begin, Time_Customer_Waiting_in_Queue, Time_Service_Ends, Time_Customer_Spend_in_System, System_idle]

  length = len(column_names)

  for i in range(length):
    x.add_column(column_names[i],data[i])

  #print(x)

  '''
  Performance measure

  Average waiting time = Total time customer wait in queue (minutes) / total number of customers

  Probability of customer (Wait) = Number of customer who wait / total number of customer

  Probability of Idle server =  Total Idle time of  server /  total  runtime of simulation

  Average time between arrival = sum of all time times between arrival / number of arrivals -1

  Average waiting time those who wait = total time customers wait in the queue / total no. of customer who wait

  Average time customer spent in the system  = total time customers customer spent in the system / total no. of customer
  '''

  # Average waiting time
  Average_waiting_time = sum(Time_Customer_Waiting_in_Queue)/size

  # Probability of customer were waiting
  no_customer_who_are_waiting = len(list(filter(lambda x:x>0,Time_Customer_Waiting_in_Queue)))

  prob_customer_waiting = no_customer_who_are_waiting / size

  # Average service time
  Average_service_time = sum(service_times)/size

  # Probability of idle server
  prob_idle_server = sum(System_idle) / Time_Service_Ends[size-1]

  # Average time between arrival
  Average_Time_Between_Arrival = arrival_time[size-1] / (len(arrival_time) - 1)

  # Average waiting time those who wait
  average_waiting_time = sum(Time_Customer_Waiting_in_Queue) / no_customer_who_are_waiting

  # Average time customer spent in the system
  time_customer_spent = sum(Time_Customer_Spend_in_System)/size

  total_idle = sum(System_idle)

  #accumulate fire.size
  fire_sizes = [fire.size for fire in fires]
  area_burned = sum(fire_sizes)

  results = {
    "Average_waiting_time": Average_waiting_time,
    "Average_service_time": Average_service_time,
    "Average_time_between_arrivals": Average_Time_Between_Arrival,
    "Average_time_customer_spent_in_system": time_customer_spent,
    "Total_area_burned": area_burned,
    "Total_idle": total_idle
  }
  return results





In [ ]:
import numpy as np

def calculate_pareto_percentiles(a, m, percentiles):
    """
    Calculate percentiles for a Pareto distribution.

    Parameters:
        a (float): Shape parameter of the Pareto distribution.
        m (float): Scale parameter of the Pareto distribution.
        percentiles (list of float): List of percentiles to calculate (values between 0 and 1).

    Returns:
        dict: Percentiles and their corresponding values.
    """
    results = {}
    for p in percentiles:
        value = m * ((1 / (1 - p)) ** (1 / a))
        results[p] = value
    return results

# Define parameters for the Pareto distribution
a = 0.42  # Shape parameter
m = 2.10  # Scale parameter

# Percentiles to calculate
percentiles = [0.60, 0.95, 0.99]

# Calculate percentiles
results = calculate_pareto_percentiles(a, m, percentiles)
print(results)
# Print results
print("Pareto Distribution Percentiles:")
for p, value in results.items():
    print(f"{int(p * 100)}th Percentile: {value:.2f}")


{0.6: 18.607820408609825, 0.95: 2629.7215213403665, 0.99: 121373.55056721934}
Pareto Distribution Percentiles:
60th Percentile: 18.61
95th Percentile: 2629.72
99th Percentile: 121373.55


In [ ]:
'''
3 Queue with jumping
'''
def sim_fires_queue_jumping(size, fires,inter_arrival_time, threshold_small, threshold_medium):
  # Calculate arrival time
  arrival_time= [0 for i in range(size)]

  for i in range(1,len(arrival_time)):
      arrival_time[i] = inter_arrival_time[i]+arrival_time[i-1]
  fire_sizes = [fire.size for fire in fires]

  arrival_time_s = []
  arrival_time_m = []
  arrival_time_l = []

  service_time_s = []
  service_time_m = []
  service_time_l = []

  Time_Customer_Waiting_in_Queue_l = []
  Time_Customer_Waiting_in_Queue_m = []
  Time_Customer_Waiting_in_Queue_s = []

  Time_Customer_Spend_in_System_l = []
  Time_Customer_Spend_in_System_m = []
  Time_Customer_Spend_in_System_s = []

  System_idle_l = []
  System_idle_m = []
  System_idle_s = []

  fire_large = []
  fire_medium = []
  fire_small = []

  i = 0
  for i, fire in enumerate(fires):
      if fire.size <= threshold_small:
          fire_small.append(fire)
          arrival_time_s.append(arrival_time[i])
          #service_time_s.append(fire.get_service_time())
      elif fire.size <= threshold_medium:
          fire_medium.append(fire)
          arrival_time_m.append(fire.get_service_time())
          #service_time_m.append(fire.get_service_time())
      else:
          fire_large.append(fire)
          arrival_time_l.append(arrival_time[i])
          #service_time_l.append(fire.get_service_time())
  # initial
  size_s = len(fire_small)
  size_m = len(fire_medium)
  size_l = len(fire_large)


  Time_Service_Begin_l = [0 for i in range(size_l)] if size_l else []
  Time_Customer_Waiting_in_Queue_l = [0 for i in range(size_l)] if size_l else []
  Time_Service_Ends_l = [0 for i in range(size_l)] if size_l else []
  Time_Customer_Spend_in_System_l = [0 for i in range(size_l)] if size_l else []
  System_idle_l = [0 for i in range(size_l)] if size_l else []

  if size_l:
    Time_Service_Begin_l[0] = arrival_time_l[0]
    Time_Service_Ends_l[0] = fire_large[0].get_service_time()
    Time_Customer_Spend_in_System_l[0] = fire_large[0].get_service_time()

  if(size_l):
    for i in range(1,size_l):
      # Time Service Begin
      Time_Service_Begin_l[i] = max(arrival_time_l[i],Time_Service_Ends_l[i-1])

      # Time customer waiting in queue
      Time_Customer_Waiting_in_Queue_l[i] = Time_Service_Begin_l[i]-arrival_time_l[i]

      if(Time_Customer_Waiting_in_Queue_l[i] > 0):
        fire_large[i].scale_fire_size(Time_Customer_Waiting_in_Queue_l[i])

      # Reduce service time to meet threshold medium/2
      fire_size_reduction = ((threshold_medium/2) / fire_large[i].size)

      # Service time used
      cur_svc_time = fire_large[i].get_service_time()*(1 - fire_size_reduction )

      # Time service ends
      Time_Service_Ends_l[i] = Time_Service_Begin_l[i] + (cur_svc_time)

      service_time_l.append(cur_svc_time)

      arrival_time_m.append(Time_Service_Ends_l[i])

      fire_medium.append(FireEvent(fire_large[i].size*fire_size_reduction,fire_large[i].intensity))

      size_m += 1

      # if size_m == 1:
      #   print("Setting initials")
      #   Time_Service_Begin_m.append(arrival_time_m[0])
      #   Time_Service_Ends_m.append(fire_medium[0].get_service_time())
      #   Time_Customer_Spend_in_System_m.append(fire_medium[0].get_service_time())
      #   Time_Customer_Waiting_in_Queue_m.append(0)
      #   System_idle_m.append(Time_Service_Ends_m[0])


      # Time Customer Spend in the system
      Time_Customer_Spend_in_System_l[i] = Time_Service_Ends_l[i] - arrival_time_l[i]

      # Time when system remains idle
      if (arrival_time_l[i]>Time_Service_Ends_l[i-1]):
        System_idle_l[i] = arrival_time_l[i]-Time_Service_Ends_l[i-1]
      else:
        System_idle_l[i] = 0

  if(size_m):
    Time_Service_Begin_m = [0 for i in range(size_m+size_l)] if size_m else []
    Time_Customer_Waiting_in_Queue_m = [0 for i in range(size_m+size_l)] if size_m else []
    Time_Service_Ends_m = [0 for i in range(size_m+size_l)] if size_m else []
    Time_Customer_Spend_in_System_m = [0 for i in range(size_m+size_l)] if size_m else []
    System_idle_m = [0 for i in range(size_m+size_l)] if size_m else []

    Time_Service_Begin_m[0] = arrival_time_m[0]
    Time_Service_Ends_m[0] = fire_medium[0].get_service_time()
    Time_Customer_Spend_in_System_m[0] = fire_medium[0].get_service_time()

    for i in range(1,size_m):
      # Time Service Begin
      Time_Service_Begin_m[i] = max(arrival_time_m[i],Time_Service_Ends_m[i-1])

      # Time customer waiting in queue
      Time_Customer_Waiting_in_Queue_m[i] = Time_Service_Begin_m[i]-arrival_time_m[i]

      fire_medium[i].scale_fire_size(Time_Customer_Waiting_in_Queue_m[i])

      # Reduce service time to meet threshold small/2
      fire_size_reduction = ((threshold_small/2) / fire_medium[i].size)


      # Service time used
      cur_svc_time = fire_medium[i].get_service_time()*(1 - fire_size_reduction )

      # Time service ends
      Time_Service_Ends_m[i] = Time_Service_Begin_m[i] + (cur_svc_time)

      service_time_m.append(cur_svc_time)

      arrival_time_s.append(Time_Service_Ends_m[i])
      fire_small.append(FireEvent(fire_medium[i].size*fire_size_reduction,fire_small[i].intensity))


      size_s += 1
      # Time Customer Spend in the system
      Time_Customer_Spend_in_System_m[i] = Time_Service_Ends_m[i] - arrival_time_m[i]

      # Time when system remains idle
      if (arrival_time_m[i]>Time_Service_Ends_m[i-1]):
        System_idle_m[i] = arrival_time_m[i]-Time_Service_Ends_m[i-1]
      else:
        System_idle_m[i] = 0
  if(size_s):
    Time_Service_Begin_s = [0 for i in range(size_s+size_m+size_l)] if size_s else []
    Time_Customer_Waiting_in_Queue_s = [0 for i in range(size_s+size_m+size_l)] if size_s else []
    Time_Service_Ends_s = [0 for i in range(size_s+size_m+size_l)] if size_s else []
    Time_Customer_Spend_in_System_s = [0 for i in range(size_s+size_m+size_l)] if size_s else []
    System_idle_s = [0 for i in range(size_s+size_m+size_l)] if size_s else []

    Time_Service_Begin_s[0] = arrival_time_s[0]
    Time_Service_Ends_s[0] = fire_small[0].get_service_time()
    Time_Customer_Spend_in_System_s[0] = fire_small[0].get_service_time()

    for i in range(1,size_s):
      # Time Service Begin
      Time_Service_Begin_s[i] = max(arrival_time_s[i],Time_Service_Ends_s[i-1])

      # Time customer waiting in queue
      Time_Customer_Waiting_in_Queue_s[i] = Time_Service_Begin_s[i]-arrival_time_s[i]

      fire_small[i].scale_fire_size(Time_Customer_Waiting_in_Queue_s[i])

      cur_svc_time = fire_small[i].get_service_time()

      # Time service ends
      Time_Service_Ends_s[i] = Time_Service_Begin_s[i] + cur_svc_time

      service_time_m.append(cur_svc_time)

      # Time Customer Spend in the system
      Time_Customer_Spend_in_System_s[i] = Time_Service_Ends_s[i] - arrival_time_s[i]

      # Time when system remains idle
      if (arrival_time_s[i]>Time_Service_Ends_s[i-1]):
        System_idle_s[i] = arrival_time_s[i]-Time_Service_Ends_s[i-1]
      else:
        System_idle_s[i] = 0

  # Average waiting time
  Average_waiting_time = (sum(Time_Customer_Waiting_in_Queue_s)+ sum(Time_Customer_Waiting_in_Queue_m) + sum(Time_Customer_Waiting_in_Queue_l))/size

  # Average service time
  Average_service_time = (sum(service_time_s) + sum(service_time_m) + sum(service_time_l))/size

  # Average time between arrival
  Average_Time_Between_Arrival = arrival_time[size-1] / (len(arrival_time) - 1)

  # Average time customer spent in the system
  time_customer_spent = (sum(Time_Customer_Spend_in_System_s)+sum(Time_Customer_Spend_in_System_m)+sum(Time_Customer_Spend_in_System_l))/size

  total_idle = sum(System_idle_s) + sum(System_idle_m) + sum(System_idle_l)

  #accumulate fire.size
  fire_sizes = [fire.size for fire in fires]
  area_burned = sum(fire_sizes)

  results = {
    "Average_waiting_time": Average_waiting_time,
    "Average_service_time": Average_service_time,
    "Average_time_between_arrivals": Average_Time_Between_Arrival,
    "Average_time_customer_spent_in_system": time_customer_spent,
    "Total_area_burned": area_burned,
    "Total_idle": total_idle
  }

  return results


In [ ]:

'''
3 Queue no jumping
'''
def sim_fires_queue_no_jumping(size, fires, inter_arrival_time, threshold_small, threshold_medium):
  # Calculate arrival time
  arrival_time= [0 for i in range(size)]

  for i in range(1,len(arrival_time)):
      arrival_time[i] = inter_arrival_time[i]+arrival_time[i-1]

  fire_sizes = [fire.size for fire in fires]
  arrival_time_s = []
  arrival_time_m = []
  arrival_time_l = []

  service_time_s = []
  service_time_m = []
  service_time_l = []

  fire_large = []
  fire_medium = []
  fire_small = []

  Time_Customer_Waiting_in_Queue_l = []
  Time_Customer_Waiting_in_Queue_m = []
  Time_Customer_Waiting_in_Queue_s = []

  Time_Customer_Spend_in_System_l = []
  Time_Customer_Spend_in_System_m = []
  Time_Customer_Spend_in_System_s = []

  System_idle_l = []
  System_idle_m = []
  System_idle_s = []

  i = 0
  for i, fire in enumerate(fires):
      #if fire.size <= threshold_small:
      if fire.size <= threshold_small: # Expected value for 60th percentile
          fire_small.append(fire)
          arrival_time_s.append(arrival_time[i])
          service_time_s.append(fire.get_service_time())

      #elif fire.size <= threshold_medium:
      elif fire.size <= threshold_medium: # Expected value for 95th percentile
          fire_medium.append(fire)
          arrival_time_m.append(fire.get_service_time())
          service_time_m.append(fire.get_service_time())
      else:
          fire_large.append(fire)
          arrival_time_l.append(arrival_time[i])
          service_time_l.append(fire.get_service_time())
  # initial
  size_s = len(fire_small)
  size_m = len(fire_medium)
  size_l = len(fire_large)

  if(size_s):
    Time_Service_Begin_s = [0 for i in range(size_s)]
    Time_Customer_Waiting_in_Queue_s = [0 for i in range(size_s)]
    Time_Service_Ends_s = [0 for i in range(size_s)]
    Time_Customer_Spend_in_System_s = [0 for i in range(size_s)]
    System_idle_s = [0 for i in range(size_s)]
  if(size_m):
    Time_Service_Begin_m = [0 for i in range(size_m)]
    Time_Customer_Waiting_in_Queue_m = [0 for i in range(size_m)]
    Time_Service_Ends_m = [0 for i in range(size_m)]
    Time_Customer_Spend_in_System_m = [0 for i in range(size_m)]
    System_idle_m = [0 for i in range(size_m)]
  if(size_l):
    Time_Service_Begin_l = [0 for i in range(size_l)]
    Time_Customer_Waiting_in_Queue_l = [0 for i in range(size_l)]
    Time_Service_Ends_l = [0 for i in range(size_l)]
    Time_Customer_Spend_in_System_l = [0 for i in range(size_l)]
    System_idle_l = [0 for i in range(size_l)]

  if(size_s):
    Time_Service_Begin_s[0] = arrival_time_s[0]
    Time_Service_Ends_s[0] = fire_small[0].get_service_time()
    Time_Customer_Spend_in_System_s[0] = fire_small[0].get_service_time()
  if(size_m):
    Time_Service_Begin_m[0] = arrival_time_m[0]
    Time_Service_Ends_m[0] = fire_medium[0].get_service_time()
    Time_Customer_Spend_in_System_m[0] = fire_medium[0].get_service_time()

  if(size_l):
    Time_Service_Begin_l[0] = arrival_time_l[0]
    Time_Service_Ends_l[0] = fire_large[0].get_service_time()
    Time_Customer_Spend_in_System_l[0] = fire_large[0].get_service_time()

    for i in range(1,size_l):
      # Time Service Begin
      Time_Service_Begin_l[i] = max(arrival_time_l[i],Time_Service_Ends_l[i-1])

      # Time customer waiting in queue
      Time_Customer_Waiting_in_Queue_l[i] = Time_Service_Begin_l[i]-arrival_time_l[i]

      if(Time_Customer_Waiting_in_Queue_l[i] > 0):
        fire_large[i].scale_fire_size(Time_Customer_Waiting_in_Queue_l[i])

      # Time service ends
      Time_Service_Ends_l[i] = Time_Service_Begin_l[i] + fire_large[i].get_service_time()

      # Time Customer Spend in the system
      Time_Customer_Spend_in_System_l[i] = Time_Service_Ends_l[i] - arrival_time_l[i]

      # Time when system remains idle
      if (arrival_time_l[i]>Time_Service_Ends_l[i-1]):
        System_idle_l[i] = arrival_time_l[i]-Time_Service_Ends_l[i-1]
      else:
        System_idle_l[i] = 0

  if(size_m):
    for i in range(1,size_m):
      # Time Service Begin
      Time_Service_Begin_m[i] = max(arrival_time_m[i],Time_Service_Ends_m[i-1])

      # Time customer waiting in queue
      Time_Customer_Waiting_in_Queue_m[i] = Time_Service_Begin_m[i]-arrival_time_m[i]

      if(Time_Customer_Waiting_in_Queue_m[i] > 0):
        fire_medium[i].scale_fire_size(Time_Customer_Waiting_in_Queue_m[i])

      # Time service ends
      Time_Service_Ends_m[i] = Time_Service_Begin_m[i] + fire_medium[i].get_service_time()

      # Time Customer Spend in the system
      Time_Customer_Spend_in_System_m[i] = Time_Service_Ends_m[i] - arrival_time_m[i]

      # Time when system remains idle
      if (arrival_time_m[i]>Time_Service_Ends_m[i-1]):
        System_idle_m[i] = arrival_time_m[i]-Time_Service_Ends_m[i-1]
      else:
        System_idle_m[i] = 0
  if(size_s):
    for i in range(1,size_s):
      # Time Service Begin
      Time_Service_Begin_s[i] = max(arrival_time_s[i],Time_Service_Ends_s[i-1])

      # Time customer waiting in queue
      Time_Customer_Waiting_in_Queue_s[i] = Time_Service_Begin_s[i]-arrival_time_s[i]

      if(Time_Customer_Waiting_in_Queue_s[i] > 0):
        fire_small[i].scale_fire_size(Time_Customer_Waiting_in_Queue_s[i])

      # Time service ends
      Time_Service_Ends_s[i] = Time_Service_Begin_s[i] + fire_small[i].get_service_time()

      # Time Customer Spend in the system
      Time_Customer_Spend_in_System_s[i] = Time_Service_Ends_s[i] - arrival_time_s[i]

      # Time when system remains idle
      if (arrival_time_s[i]>Time_Service_Ends_s[i-1]):
        System_idle_s[i] = arrival_time_s[i]-Time_Service_Ends_s[i-1]
      else:
        System_idle_s[i] = 0

  # Average waiting time
  Average_waiting_time = (sum(Time_Customer_Waiting_in_Queue_s)+ sum(Time_Customer_Waiting_in_Queue_m) + sum(Time_Customer_Waiting_in_Queue_l))/size

  service_time_s = [fire.get_service_time() for fire in fire_small]

  service_time_m = [fire.get_service_time() for fire in fire_medium]

  service_time_l = [fire.get_service_time() for fire in fire_large]

  # Average service time
  Average_service_time = (sum(service_time_s) + sum(service_time_m) + sum(service_time_l))/size

  # Average time between arrival
  Average_Time_Between_Arrival = arrival_time[size-1] / (len(arrival_time) - 1)

  # Average time customer spent in the system
  time_customer_spent = (sum(Time_Customer_Spend_in_System_s)+sum(Time_Customer_Spend_in_System_m)+sum(Time_Customer_Spend_in_System_l))/size

  #accumulate fire.size
  fire_sizes = [fire.size for fire in fires]

  area_burned = sum(fire_sizes)

  total_idle = sum(System_idle_s) + sum(System_idle_m) + sum(System_idle_l)

  #accumulate fire.size
  fire_sizes = [fire.size for fire in fires]
  area_burned = sum(fire_sizes)
  results = {
    "Average_waiting_time": Average_waiting_time,
    "Average_service_time": Average_service_time,
    "Average_time_between_arrivals": Average_Time_Between_Arrival,
    "Average_time_customer_spent_in_system": time_customer_spent,
    "Total_area_burned": area_burned,
    "Total_idle": total_idle
  }
  return results


In [ ]:

'''
3 queue even dist
'''
def sim_fires_queue_even_dist(size, fires, inter_arrival_time, threshold_small, threshold_medium):
  # Calculate arrival time
  arrival_time= [0 for i in range(size)]

  for i in range(1,len(arrival_time)):
      arrival_time[i] = inter_arrival_time[i]+arrival_time[i-1]

  #fire_sizes = [fire.size for fire in fires]

  arrival_time_q1 = []
  arrival_time_q2 = []
  arrival_time_q3 = []

  Time_Customer_Waiting_in_Queue_q1 = []
  Time_Customer_Waiting_in_Queue_q2 = []
  Time_Customer_Waiting_in_Queue_q3 = []

  Time_Customer_Spend_in_System_q1 = []
  Time_Customer_Spend_in_System_q2 = []
  Time_Customer_Spend_in_System_q3 = []

  fire_q1 = []
  fire_q2 = []
  fire_q3 = []

  fire_size_counts_q1 = {"small": 0, "medium": 0, "large": 0}

  fire_size_counts_q2 = {"small": 0, "medium": 0, "large": 0}

  fire_size_counts_q3 = {"small": 0, "medium": 0, "large": 0}

  i = 0
  for i, fire in enumerate(fires):
    if fire.size <= threshold_small: # Expected value for 60th percentile
        if fire_size_counts_q1["small"] == fire_size_counts_q2["small"] and fire_size_counts_q1["small"] == fire_size_counts_q3["small"]:
            fire_q1.append(fire)
            arrival_time_q1.append(arrival_time[i])
            fire_size_counts_q1["small"] += 1
        elif fire_size_counts_q2["small"] == fire_size_counts_q3["small"]:
            fire_q2.append(fire)
            arrival_time_q2.append(arrival_time[i])
            fire_size_counts_q2["small"] += 1
        else:
            fire_q3.append(fire)
            arrival_time_q3.append(arrival_time[i])
            fire_size_counts_q3["small"] += 1
    elif fire.size <= threshold_medium: # Expected value for 95th percentile
        if fire_size_counts_q1["medium"] == fire_size_counts_q2["medium"] and fire_size_counts_q1["medium"] == fire_size_counts_q3["medium"]:
            fire_q1.append(fire)
            arrival_time_q1.append(arrival_time[i])
            fire_size_counts_q1["medium"] += 1
        elif fire_size_counts_q2["medium"] == fire_size_counts_q3["medium"]:
            fire_q2.append(fire)
            arrival_time_q2.append(arrival_time[i])
            fire_size_counts_q2["medium"] += 1
        else:
            fire_q3.append(fire)
            arrival_time_q3.append(arrival_time[i])
            fire_size_counts_q3["medium"] += 1
    else: # Large Fires
        if fire_size_counts_q1["large"] == fire_size_counts_q2["large"] and fire_size_counts_q1["large"] == fire_size_counts_q3["large"]:
            fire_q1.append(fire)
            arrival_time_q1.append(arrival_time[i])
            fire_size_counts_q1["large"] += 1
        elif fire_size_counts_q2["large"] == fire_size_counts_q3["large"]:
            fire_q2.append(fire)
            arrival_time_q2.append(arrival_time[i])
            fire_size_counts_q2["large"] += 1
        else:
            fire_q3.append(fire)
            arrival_time_q3.append(arrival_time[i])
            fire_size_counts_q3["large"] += 1

  # initial
  size_q1 = len(fire_q1)
  size_q2 = len(fire_q2)
  size_q3 = len(fire_q3)

  Time_Service_Begin_q1 = [0 for i in range(size_q1)]
  Time_Customer_Waiting_in_Queue_q1 = [0 for i in range(size_q1)]
  Time_Service_Ends_q1 = [0 for i in range(size_q1)]
  Time_Customer_Spend_in_System_q1 = [0 for i in range(size_q1)]
  System_idle_q1 = [0 for i in range(size_q1)]

  Time_Service_Begin_q2 = [0 for i in range(size_q2)]
  Time_Customer_Waiting_in_Queue_q2 = [0 for i in range(size_q2)]
  Time_Service_Ends_q2 = [0 for i in range(size_q2)]
  Time_Customer_Spend_in_System_q2 = [0 for i in range(size_q2)]
  System_idle_q2 = [0 for i in range(size_q2)]

  Time_Service_Begin_q3 = [0 for i in range(size_q3)]
  Time_Customer_Waiting_in_Queue_q3 = [0 for i in range(size_q3)]
  Time_Service_Ends_q3 = [0 for i in range(size_q3)]
  Time_Customer_Spend_in_System_q3 = [0 for i in range(size_q3)]
  System_idle_q3 = [0 for i in range(size_q3)]

  Time_Service_Begin_q1[0] = arrival_time_q1[0]
  Time_Service_Ends_q1[0] = fire_q1[0].get_service_time()
  Time_Customer_Spend_in_System_q1[0] = fire_q1[0].get_service_time()

  Time_Service_Begin_q2[0] = arrival_time_q2[0]
  Time_Service_Ends_q2[0] = fire_q2[0].get_service_time()
  Time_Customer_Spend_in_System_q2[0] = fire_q2[0].get_service_time()

  Time_Service_Begin_q3[0] = arrival_time_q3[0]
  Time_Service_Ends_q3[0] = fire_q3[0].get_service_time()
  Time_Customer_Spend_in_System_q3[0] = fire_q3[0].get_service_time()

  for i in range(1,size_q1):
    # Time Service Begin
    Time_Service_Begin_q1[i] = max(arrival_time_q1[i],Time_Service_Ends_q1[i-1])

    # Time customer waiting in queue
    Time_Customer_Waiting_in_Queue_q1[i] = Time_Service_Begin_q1[i]-arrival_time_q1[i]

    if(Time_Customer_Waiting_in_Queue_q1[i] > 0):
      fire_q1[i].scale_fire_size(Time_Customer_Waiting_in_Queue_q1[i])

    # Time service ends
    Time_Service_Ends_q1[i] = Time_Service_Begin_q1[i] + fire_q1[i].get_service_time()

    # Time Customer Spend in the system
    Time_Customer_Spend_in_System_q1[i] = Time_Service_Ends_q1[i] - arrival_time_q1[i]

    # Time when system remains idle
    if (arrival_time_q1[i]>Time_Service_Ends_q1[i-1]):
      System_idle_q1[i] = arrival_time_q1[i]-Time_Service_Ends_q1[i-1]
    else:
      System_idle_q1[i] = 0

  for i in range(1,size_q2):
    # Time Service Begin
    Time_Service_Begin_q2[i] = max(arrival_time_q2[i],Time_Service_Ends_q2[i-1])

    # Time customer waiting in queue
    Time_Customer_Waiting_in_Queue_q2[i] = Time_Service_Begin_q2[i]-arrival_time_q2[i]

    if(Time_Customer_Waiting_in_Queue_q2[i] > 0):
      fire_q2[i].scale_fire_size(Time_Customer_Waiting_in_Queue_q2[i])

    # Time service ends
    Time_Service_Ends_q2[i] = Time_Service_Begin_q2[i] + fire_q2[i].get_service_time()

    # Time Customer Spend in the system
    Time_Customer_Spend_in_System_q2[i] = Time_Service_Ends_q2[i] - arrival_time_q2[i]

    # Time when system remains idle
    if (arrival_time_q2[i]>Time_Service_Ends_q2[i-1]):
      System_idle_q2[i] = arrival_time_q2[i]-Time_Service_Ends_q2[i-1]
    else:
      System_idle_q2[i] = 0

  for i in range(1,size_q3):
    # Time Service Begin
    Time_Service_Begin_q3[i] = max(arrival_time_q3[i],Time_Service_Ends_q3[i-1])

    # Time customer waiting in queue
    Time_Customer_Waiting_in_Queue_q3[i] = Time_Service_Begin_q3[i]-arrival_time_q3[i]

    if(Time_Customer_Waiting_in_Queue_q3[i] > 0):
      fire_q3[i].scale_fire_size(Time_Customer_Waiting_in_Queue_q3[i])

    # Time service ends
    Time_Service_Ends_q3[i] = Time_Service_Begin_q3[i] + fire_q3[i].get_service_time()

    # Time Customer Spend in the system
    Time_Customer_Spend_in_System_q3[i] = Time_Service_Ends_q3[i] - arrival_time_q3[i]

    # Time when system remains idle
    if (arrival_time_q3[i]>Time_Service_Ends_q3[i-1]):
      System_idle_q3[i] = arrival_time_q3[i]-Time_Service_Ends_q3[i-1]
    else:
      System_idle_q3[i] = 0

  # Average waiting time
  Average_waiting_time = (sum(Time_Customer_Waiting_in_Queue_q1)+ sum(Time_Customer_Waiting_in_Queue_q2) + sum(Time_Customer_Waiting_in_Queue_q3))/size

  service_time_q3 = [fire.get_service_time() for fire in fire_q3]

  service_time_q2 = [fire.get_service_time() for fire in fire_q2]

  service_time_q1 = [fire.get_service_time() for fire in fire_q1]

  # Average service time
  Average_service_time = (sum(service_time_q1) + sum(service_time_q2) + sum(service_time_q3))/size

  # Average time between arrival
  Average_Time_Between_Arrival = arrival_time[size-1] / (len(arrival_time) - 1)

  # Average time customer spent in the system
  time_customer_spent = (sum(Time_Customer_Spend_in_System_q1)+sum(Time_Customer_Spend_in_System_q2)+sum(Time_Customer_Spend_in_System_q3))/size

  #accumulate fire.size
  fire_sizes = [fire.size for fire in fires]

  area_burned = sum(fire_sizes)

  total_idle = sum(System_idle_q1) + sum(System_idle_q2) + sum(System_idle_q3)

  #accumulate fire.size
  fire_sizes = [fire.size for fire in fires]
  area_burned = sum(fire_sizes)
  results = {
    "Average_waiting_time": Average_waiting_time,
    "Average_service_time": Average_service_time,
    "Average_time_between_arrivals": Average_Time_Between_Arrival,
    "Average_time_customer_spent_in_system": time_customer_spent,
    "Total_area_burned": area_burned,
    "Total_idle": total_idle,
  }
  return results


In [ ]:
# No. of Customer
size = 100

fires, inter_arrival_time = generate_fire_events(size)


# percentiles = np.percentile(fire_sizes, [60, 95])  # 60% and 95% thresholds

# threshold_small, threshold_medium = percentiles

percentiles = calculate_pareto_percentiles(0.42, 2.10, [0.60, 0.95])
threshold_small = percentiles[0.60]
threshold_medium = percentiles[0.95]

val = sim_fires_queue_even_dist(size, fires, inter_arrival_time, threshold_small, threshold_medium)
#print(val)

In [ ]:
import matplotlib.pyplot as plt

percentiles_list = [
    [0.60, 0.70],
    [0.60, 0.75],
    [0.65, 0.75],
    [0.60, 0.80],
    [0.65, 0.80],
    [0.70, 0.80],
    [0.50, 0.85],
    [0.70, 0.85],
    [0.75, 0.85],
    [0.80, 0.90],
    [0.75, 0.90],
    [0.75, 0.90],
    [0.40, 0.94],
    [0.50, 0.94],
    [0.60, 0.94],
    [0.70, 0.94],
    [0.80, 0.94],
    [0.80, 0.95],
    [0.85, 0.95],
    [0.90, 0.95],
    [0.80, 0.975],
    [0.85, 0.975],
    [0.90, 0.975],
    [0.92, 0.975],
    [0.75, 0.975],
    [0.80, 0.99],
    [0.85, 0.99],
    [0.90, 0.99],
    [0.95, 0.99],
]


# Placeholder to store the results of the simulations
all_results = []
size = 100
seeds = range(1, 110)  # Range of seeds

# Initialize variables to track the best configuration
best_result = None
best_score = float('inf')  # Start with a very high score

# Iterate through each set of percentiles
for percentiles in percentiles_list:
    scores = []  # Store scores for each seed

    for seed in seeds:
        # Generate fires and inter-arrival times
        fires, inter_arrival_time = generate_fire_events(size, seed)

        # Calculate arrival times
        arrival_time = [0] * size
        for i in range(1, size):
            arrival_time[i] = inter_arrival_time[i] + arrival_time[i - 1]

        # Calculate thresholds for the current percentiles
        thresholds = calculate_pareto_percentiles(0.42, 2.10, percentiles)
        threshold_small = thresholds[percentiles[0]]
        threshold_medium = thresholds[percentiles[1]]

        # Simulate the fire queue
        #sim_results = sim_fires_queue_no_jumping(size, fires,inter_arrival_time, threshold_small, threshold_medium,)
        #sim_results = sim_fires_queue_jumping(size, fires,inter_arrival_time, threshold_small, threshold_medium)

        sim_results = sim_fires_queue_even_dist(size, fires, inter_arrival_time, threshold_small, threshold_medium)

        # Extract key metrics for evaluation
        scores.append(sim_results["Total_area_burned"])

    # Average score across seeds
    avg_score = sum(scores) / len(scores)

    # Store the result with its associated metrics
    all_results.append({
        "percentiles": percentiles,
        "thresholds": (threshold_small, threshold_medium),
        "average_score": avg_score,
        "scores": scores,
    })

    # Update the best result based on the average score
    if avg_score < best_score:
        best_score = avg_score
        best_result = {
            "percentiles": percentiles,
            "thresholds": (threshold_small, threshold_medium),
            "average_score": avg_score,
            "scores": scores,
        }

# Output the most effective configuration
print("\nMost Effective Configuration:")
print(f"Percentiles: {best_result['percentiles']}")
print(f"Thresholds: Small={best_result['thresholds'][0]:.2f}, Medium={best_result['thresholds'][1]:.2f}")
print(f"Average Score (Total Area Burned): {best_result['average_score']:.2f}")

# Visualization of results
percentiles_labels = [f"{result['percentiles'][0]}-{result['percentiles'][1]}" for result in all_results]
average_scores = [result["average_score"] for result in all_results]

# Normalize scores for proportional visualization
max_score = max(average_scores)
normalized_scores = [score / max_score for score in average_scores]

# Plot the results
plt.figure(figsize=(12, 6))
bars = plt.bar(percentiles_labels, normalized_scores, color='skyblue', edgecolor='black')

# Highlight the best configuration
best_result_index = average_scores.index(min(average_scores))
bars[best_result_index].set_color('green')
bars[best_result_index].set_edgecolor('black')

# Add labels and title
plt.title("Proportional Performance Across Seeds", fontsize=16)
plt.xlabel("Percentile Ranges", fontsize=12)
plt.ylabel("Normalized Average Total Area Burned", fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()

# Annotate the best configuration
plt.annotate(f"Best: {best_score:.2f}",
             (best_result_index, normalized_scores[best_result_index]),
             textcoords="offset points",
             xytext=(0, 5),
             ha='center',
             fontsize=10,
             color='green',
             fontweight='bold')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Configuration for simulation
percentiles_wq = [0.95, 0.99]  # with queue optimal bucket sizes
percentiles_nq = [0.95, 0.99]  # no queue optimal bucket sizes
percentiles_ed = [0.95, 0.975]  # even distribution bucket sizes

size = 100
seeds = range(1, 1000)  # Use multiple seeds for simulation

# Placeholder for results
metrics_labels = ["With Queue Jumping", "No Queue Jumping", "FIFO", "Even Distribution"]
aggregate_metrics = {label: {"Total_area_burned": [], "Total_idle": []} for label in metrics_labels}

# Tally for the least total area burned
least_burned_tally = {label: 0 for label in metrics_labels}

for seed in seeds:
    metrics_values = {}

    # Calculate thresholds and simulate for "with queue jumping"
    thresholds = calculate_pareto_percentiles(0.42, 2.10, percentiles_wq)
    threshold_small = thresholds[percentiles_wq[0]]
    threshold_medium = thresholds[percentiles_wq[1]]

    fires, inter_arrival_time = generate_fire_events(size, seed)
    val_wq = sim_fires_queue_jumping(size, fires, inter_arrival_time, threshold_small, threshold_medium)
    metrics_values["With Queue Jumping"] = val_wq

    # Calculate thresholds and simulate for "no queue jumping"
    thresholds = calculate_pareto_percentiles(0.42, 2.10, percentiles_nq)
    threshold_small = thresholds[percentiles_nq[0]]
    threshold_medium = thresholds[percentiles_nq[1]]

    fires, inter_arrival_time = generate_fire_events(size, seed)
    val_nq = sim_fires_queue_no_jumping(size, fires, inter_arrival_time, threshold_small, threshold_medium)
    metrics_values["No Queue Jumping"] = val_nq

    # Simulate for FIFO
    fires, inter_arrival_time = generate_fire_events(size, seed)
    val_fifo = sim_fires_fifo(size, fires, inter_arrival_time)
    metrics_values["FIFO"] = val_fifo

    # Calculate thresholds and simulate for "even distribution"
    thresholds = calculate_pareto_percentiles(0.42, 2.10, percentiles_ed)
    threshold_small = thresholds[percentiles_ed[0]]
    threshold_medium = thresholds[percentiles_ed[1]]

    fires, inter_arrival_time = generate_fire_events(size, seed)
    val_ed = sim_fires_queue_even_dist(size, fires, inter_arrival_time, threshold_small, threshold_medium)
    metrics_values["Even Distribution"] = val_ed

    # Find the simulation with the least area burned for this run
    least_burned_strategy = min(metrics_values, key=lambda x: metrics_values[x]["Total_area_burned"])
    least_burned_tally[least_burned_strategy] += 1

    # Append results to aggregate metrics
    for label in metrics_labels:
        aggregate_metrics[label]["Total_area_burned"].append(metrics_values[label]["Total_area_burned"])
        aggregate_metrics[label]["Total_idle"].append(metrics_values[label]["Total_idle"])

# Calculate averages for visualization
average_metrics = {
    label: {
        "Total_area_burned": sum(values["Total_area_burned"]) / len(values["Total_area_burned"]),
        "Total_idle": sum(values["Total_idle"]) / len(values["Total_idle"]),
    }
    for label, values in aggregate_metrics.items()
}

# Normalize metrics for proportional visualization
max_burned = max(average_metrics[label]["Total_area_burned"] for label in metrics_labels)
normalized_burned_all = {label: values["Total_area_burned"] / max_burned for label, values in average_metrics.items()}

max_idle = max(average_metrics[label]["Total_idle"] for label in metrics_labels)
normalized_idle_all = {label: values["Total_idle"] / max_idle for label, values in average_metrics.items()}

# Graph: Normalized Total Area Burned for All Strategies
plt.figure(figsize=(10, 6))
bars = plt.bar(normalized_burned_all.keys(), normalized_burned_all.values(),
               color=['green', 'blue', 'orange', 'purple'], edgecolor='black')
plt.title("Normalized Total Area Burned for All Strategies", fontsize=16)
plt.ylabel("Normalized Total Area Burned", fontsize=12)
for i, (label, value) in enumerate(normalized_burned_all.items()):
    plt.text(i, value + 0.02, f"{average_metrics[label]['Total_area_burned']:.2f}", ha='center', fontsize=10)
plt.show()
plt.close()

# FIFO vs Even Distribution comparison
fifo_ed_labels = ["FIFO", "Even Distribution"]
fifo_ed_burned = [
    average_metrics["FIFO"]["Total_area_burned"],
    average_metrics["Even Distribution"]["Total_area_burned"],
]
fifo_ed_idle = [
    average_metrics["FIFO"]["Total_idle"],
    average_metrics["Even Distribution"]["Total_idle"],
]

# Normalization for proportional visualization
max_burned = max(fifo_ed_burned)
normalized_burned = [value / max_burned for value in fifo_ed_burned]

max_idle = max(fifo_ed_idle)
normalized_idle = [value / max_idle for value in fifo_ed_idle]

# Chart 1: Total Area Burned for FIFO vs Even Distribution
plt.figure(figsize=(10, 6))
bars = plt.bar(fifo_ed_labels, normalized_burned, color=['orange', 'purple'], edgecolor='black')
plt.title("Normalized Total Area Burned: FIFO vs Even Distribution", fontsize=16)
plt.ylabel("Normalized Total Area Burned", fontsize=12)
for i, value in enumerate(fifo_ed_burned):
    plt.text(i, normalized_burned[i] + 0.02, f"{value:.2f}", ha='center', fontsize=10)
plt.show()
plt.close()

# Chart 2: Total Idle Time for FIFO vs Even Distribution
plt.figure(figsize=(10, 6))
bars = plt.bar(fifo_ed_labels, normalized_idle, color=['orange', 'purple'], edgecolor='black')
plt.title("Normalized Total Idle Time: FIFO vs Even Distribution", fontsize=16)
plt.ylabel("Normalized Total Idle Time", fontsize=12)
for i, value in enumerate(fifo_ed_idle):
    plt.text(i, normalized_idle[i] + 0.02, f"{value:.2f}", ha='center', fontsize=10)
plt.show()
plt.close()


In [438]:
import random
import copy

# Configuration for simulation
percentiles_wq = [0.95, 0.99]  # with queue optimal bucket sizes
percentiles_nq = [0.95, 0.99]  # no queue optimal bucket sizes
percentiles_ed = [0.95, 0.975]  # even distribution bucket sizes

size = 100
num_runs = 5  # Number of runs for the simulation

# Placeholder for results
metrics_labels = ["With Queue Jumping", "No Queue Jumping", "FIFO", "Even Distribution"]

# Simulate and print results
for run in range(1, num_runs + 1):
    print(f"\n=== Simulation Run {run} ===")

    results = {}

    # Use a fixed seed for common random numbers
    seed = random.randint(1, 1000)
    random.seed(seed)

    # Generate fires and inter-arrival times once per run
    fires, inter_arrival_time = generate_fire_events(size, seed)

    # Calculate thresholds and simulate for each strategy
    for label, percentiles in zip(metrics_labels, [percentiles_wq, percentiles_nq, None, percentiles_ed]):
        thresholds = (
            calculate_pareto_percentiles(0.42, 2.10, percentiles)
            if percentiles else None
        )
        threshold_small = thresholds[percentiles[0]] if thresholds else None
        threshold_medium = thresholds[percentiles[1]] if thresholds else None

        if label == "With Queue Jumping":
            results[label] = sim_fires_queue_jumping(size, copy.deepcopy(fires), copy.deepcopy(inter_arrival_time), threshold_small, threshold_medium)
        elif label == "No Queue Jumping":
            results[label] = sim_fires_queue_no_jumping(size, copy.deepcopy(fires), copy.deepcopy(inter_arrival_time), threshold_small, threshold_medium)
        elif label == "FIFO":
            results[label] = sim_fires_fifo(size, copy.deepcopy(fires), copy.deepcopy(inter_arrival_time))
        elif label == "Even Distribution":
            results[label] = sim_fires_queue_even_dist(size, copy.deepcopy(fires), copy.deepcopy(inter_arrival_time), threshold_small, threshold_medium)

    # Print formatted results for this run
    for label in metrics_labels:
        result = results[label]
        print(f"--- {label} ---")
        print(f"Average Waiting Time: {result['Average_waiting_time']:.2f}")
        print(f"Average Service Time: {result['Average_service_time']:.2f}")
        print(f"Average Time Between Arrivals: {result['Average_time_between_arrivals']:.2f}")
        print(f"Average Time Customer Spent in System: {result['Average_time_customer_spent_in_system']:.2f}")
        print(f"Total Area Burned: {result['Total_area_burned']:.2f}")

    print("=== End of Run ===")



=== Simulation Run 1 ===
--- With Queue Jumping ---
Average Waiting Time: 268.16
Average Service Time: 119.29
Average Time Between Arrivals: 0.23
Average Time Customer Spent in System: 388.59
Total Area Burned: 862421.20
--- No Queue Jumping ---
Average Waiting Time: 270.40
Average Service Time: 121.40
Average Time Between Arrivals: 0.23
Average Time Customer Spent in System: 391.81
Total Area Burned: 868762.99
--- FIFO ---
Average Waiting Time: 4.70
Average Service Time: 29.71
Average Time Between Arrivals: 0.23
Average Time Customer Spent in System: 34.40
Total Area Burned: 223479.67
--- Even Distribution ---
Average Waiting Time: 249.11
Average Service Time: 38.92
Average Time Between Arrivals: 0.23
Average Time Customer Spent in System: 288.03
Total Area Burned: 290758.65
=== End of Run ===

=== Simulation Run 2 ===
--- With Queue Jumping ---
Average Waiting Time: 128.50
Average Service Time: 589.33
Average Time Between Arrivals: 0.26
Average Time Customer Spent in System: 760.25
